# Practica APA - predicció superfície cremada d'un incendi
Sergi Curto Panisello,
Joan Melchor Lladó

## Imports

In [5]:
import pandas as pd
import numpy as np

## Obtenim les dades
Procedim a agafar les dades en format csv descarregades de https://datos.civio.es/dataset/todos-los-incendios-forestales/

In [6]:
df = pd.read_csv('fires-all.csv')
print(df.describe())

                 id    superficie           lat           lng  \
count  8.264000e+04  82640.000000  82616.000000  82616.000000   
mean   2.007259e+09     19.888085     41.763721     -5.664360   
std    4.247106e+06    223.787536      2.019672      4.394031   
min    2.001010e+09      1.000000      0.490720  -1000.000000   
25%    2.003390e+09      1.500000     40.876544     -7.257298   
50%    2.006330e+09      3.000000     42.371123     -6.019478   
75%    2.011150e+09      7.150000     43.143145     -4.303078   
max    2.015510e+09  28879.100000     87.824157    242.755603   

       latlng_explicit   idcomunidad   idprovincia   idmunicipio  \
count     82640.000000  82640.000000  82640.000000  82640.000000   
mean          0.741626      7.850278     28.411375     77.795946   
std           0.437743      5.419922     11.727155     98.867769   
min           0.000000      1.000000      1.000000      1.000000   
25%           0.000000      3.000000     21.000000     27.000000   
50%   

## Preprocessing

### Tractament de missing values

In [7]:
# Columnes amb missing values, també es pot veure al describe del
# dataset a les variables que no tinguin un count de 82640
print(df.columns[df.isnull().any()].tolist())

['lat', 'lng', 'causa_supuesta', 'muertos', 'heridos', 'gastos', 'perdidas']


#### Arreglar lat i long

Per arreglar lat i long ho fem en un altre document "corregirCoordenades.py", ja que és un procés més lent.
Es dedica a comprovar les coordenades de cada instància per veure si es corresponen a un diccionari generat
anteriorment amb el fitxer "obtenirCoordenades.py". Hi ha una explicació més extensa a la documentació.

In [8]:
df = pd.read_csv('coordsCorregides.csv')
# Comprovem que ja no queden nulls a coordenades
print(df.columns[df.isnull().any()].tolist())

['causa_supuesta', 'muertos', 'heridos', 'gastos', 'perdidas']


In [9]:
# Comencem per emplenar els missing values de muertos i heridos ja que
# si no hi han dades suposarem que són 0.
df['muertos'].fillna(0, inplace=True)
df['heridos'].fillna(0, inplace=True)
# Sobre la "causa supuesta" és normal que hi hagin instàncies sense valor ja que és o 1 o nan per tant els posem a 0
df['causa_supuesta'].fillna(0, inplace=True)

In [10]:
# Fem knn
df['gastos'].fillna(0, inplace=True)
df['perdidas'].fillna(0, inplace=True)

# df.describe()

print(df.columns[df.isnull().any()].tolist())

[]


In [11]:
print(df.describe())

         Unnamed: 0            id    superficie           lat           lng  \
count  82527.000000  8.252700e+04  82527.000000  82527.000000  82527.000000   
mean   41310.892835  2.007258e+09     19.091822     41.719200     -5.557235   
std    23855.081820  4.246828e+06    207.080716      1.926826      2.443357   
min        0.000000  2.001010e+09      1.000000     27.707180    -17.980518   
25%    20658.500000  2.003390e+09      1.500000     40.775726     -7.248053   
50%    41304.000000  2.006330e+09      3.000000     42.342648     -5.959559   
75%    61959.500000  2.011150e+09      7.100000     43.127865     -4.217251   
max    82639.000000  2.015510e+09  28879.100000     43.761264      4.158954   

       latlng_explicit   idcomunidad   idprovincia   idmunicipio  \
count     82527.000000  82527.000000  82527.000000  82527.000000   
mean          0.640808      7.850219     28.412144     76.534879   
std           0.479766      5.419601     11.723812     92.871866   
min           0.

In [12]:
aux = df.loc[df['idmunicipio'] == 999]
latMissing = df.loc[np.isnan(df['lat'])]
print(aux.shape)
print(latMissing.shape)
print(latMissing)

(0, 22)
(0, 22)
Empty DataFrame
Columns: [Unnamed: 0, id, superficie, fecha, lat, lng, latlng_explicit, idcomunidad, idprovincia, idmunicipio, municipio, causa, causa_supuesta, causa_desc, muertos, heridos, time_ctrl, time_ext, personal, medios, gastos, perdidas]
Index: []

[0 rows x 22 columns]


## Separem target del dataset
Volem predir la superfície cremada donat un incendi per tant separem aquesta variable de la resta.

In [13]:
X = df.drop('superficie', axis=1)
y = df['superficie']

# Comprovem que al target no hi hagin nulls
print(y.isnull().values.any())


False
